# Modeling interest rate swaps using QuantLib

An interest rate swap is a financial derivative instrument in which two parties agree to exchange interest rate cash flows based on a notional amount from a fixed rate to a floating rate or from one floating rate to another floating rate. 

Here we will consider an example of a plain vanilla USD swap with 10 million notional and 10 year maturity. Let the fixed leg pay 2.5% coupon semiannually, and the floating leg pay Libor 3m quarterly. 

In [1]:
from QuantLib import *
calculation_date = Date(20, 10, 2015)
Settings.instance().evaluationDate = calculation_date

Here we construct the yield curve objects. For simplicity, we will use flat curves for discounting and Libor 3M. This will help us focus on the swap construction. Please refer to [curve construction example](http://gouthamanbalaraman.com/blog/quantlib-term-structure-bootstrap-yield-curve.html) for some details.

In [2]:
# construct discount curve and libor curve

risk_free_rate = 0.01
libor_rate = 0.02
day_count = Actual365Fixed()

discount_curve = YieldTermStructureHandle(
    FlatForward(calculation_date, risk_free_rate, day_count)
)

libor_curve = YieldTermStructureHandle(
    FlatForward(calculation_date, libor_rate, day_count)
)
#libor3M_index = ql.Euribor3M(libor_curve)  
libor3M_index = USDLibor(Period(3, Months), libor_curve)

To construct the swap, we have to specify the fixed rate leg and floating rate leg. We construct the fixed rate and floating rate leg schedules below.

In [3]:
calendar = UnitedStates()
settle_date = calendar.advance(calculation_date, 5, Days)
maturity_date = calendar.advance(settle_date, 10, Years)

fixed_leg_tenor = Period(6, Months)
fixed_schedule = Schedule(settle_date, maturity_date, 
                          fixed_leg_tenor, calendar,
                          ModifiedFollowing, ModifiedFollowing,
                          DateGeneration.Forward, False)

float_leg_tenor = Period(3, Months)
float_schedule = Schedule(settle_date, maturity_date, 
                          float_leg_tenor, calendar,
                          ModifiedFollowing, ModifiedFollowing,
                          DateGeneration.Forward, False)

Below, we construct a `VanillaSwap` object  by including the fixed and float leg schedules created above.

In [4]:
notional = 10000000
fixed_rate = 0.025
fixed_leg_daycount = Actual360()
float_spread = 0.004
float_leg_daycount = Actual360()

ir_swap = VanillaSwap(VanillaSwap.Payer, notional, fixed_schedule, 
                      fixed_rate, fixed_leg_daycount, float_schedule,
                      libor3M_index, float_spread, float_leg_daycount )

We evaluate the swap using a discounting engine.

In [5]:
swap_engine = DiscountingSwapEngine(discount_curve)
ir_swap.setPricingEngine(swap_engine)

#### Result Analysis

The cash flows for the fixed and floating leg can be extracted from the `ir_swap` object. The fixed leg cash flows are shown below:

In [6]:
from pandas import DataFrame
DataFrame(
    [(cf.date(), cf.amount()) for cf in ir_swap.leg(0)],
    columns=["Date", "Amount"],
    index=range(1, len(ir_swap.leg(0))+1))

,Date,Amount
1,"April 27th, 2016",127083.333333
2,"October 27th, 2016",127083.333333
3,"April 27th, 2017",126388.888889
4,"October 27th, 2017",127083.333333
5,"April 27th, 2018",126388.888889
6,"October 29th, 2018",128472.222222
7,"April 29th, 2019",126388.888889
8,"October 28th, 2019",126388.888889
9,"April 27th, 2020",126388.888889
10,"October 27th, 2020",127083.333333


The floating leg cash flows are shown below:

In [7]:
from pandas import DataFrame
DataFrame(
    [(cf.date(), cf.amount()) for cf in ir_swap.leg(1)],
    columns=["Date", "Amount"],
    index=range(1, len(ir_swap.leg(1))+1))

,Date,Amount
1,"January 27th, 2016",60760.458147
2,"April 27th, 2016",60098.647700
3,"July 27th, 2016",60098.647700
4,"October 27th, 2016",60760.458147
5,"January 27th, 2017",60760.458147
6,"April 27th, 2017",59436.867427
7,"July 27th, 2017",60098.647700
8,"October 27th, 2017",60760.458147
9,"January 29th, 2018",62084.169572
10,"April 27th, 2018",58113.397399


Some other analytics such as the fair value, fair spread etc can be extracted as shown below.

In [8]:
print("%-20s: %20.3f" % ("Net Present Value", ir_swap.NPV()))
print("%-20s: %20.3f" % ("Fair Spread", ir_swap.fairSpread()))
print("%-20s: %20.3f" % ("Fair Rate", ir_swap.fairRate()))
print("%-20s: %20.3f" % ("Fixed Leg BPS", ir_swap.fixedLegBPS()))
print("%-20s: %20.3f" % ("Floating Leg BPS", ir_swap.floatingLegBPS()))

Net Present Value   :          -115054.034
Fair Spread         :                0.005
Fair Rate           :                0.024
Fixed Leg BPS       :            -9629.981
Floating Leg BPS    :             9642.042


#### Conclusion

Here we saw a simple example on how to construct a swap and value them. We evaluated the fixed and floating legs and then valued the `VanillaSwap` using the `DiscountingSwapEngine`.